In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.naive_bayes import GaussianNB,BernoulliNB,ComplementNB,MultinomialNB
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression as LR
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [ ]:
def gridsearch_bulid(svc, gridsearch_para, clf_name):
    grid_cv = RepeatedKFold(n_splits=5, n_repeats=1, random_state=10)
    grid_model = copy.deepcopy(svc)
    param_grid = grid_space(svc, gridsearch_para, clf_name)
    grid_search = GridSearchCV(grid_model, param_grid, cv=grid_cv, scoring='r2', n_jobs=5)
    return grid_search

In [ ]:
def grid_space(model, gridsearch_para, clf_name):
    if clf_name == 'Naive Bayes':
        search_space = gridsearch_para
    if clf_name == 'SVM':
        search_space = {
            'C': gridsearch_para['C'],
        }
        if model.get_params()['kernel'] == 'rbf':
            search_space['gamma'] = list(gridsearch_para['gamma']) + ['scale', 'auto']
        elif model.get_params()['kernel'] == 'poly':
            search_space['gamma'] = list(gridsearch_para['gamma']) + ['scale', 'auto']
            search_space['coef0'] = gridsearch_para['coef0']
            search_space['degree'] = gridsearch_para['degree']
        elif model.get_params()['kernel'] == 'sigmoid':
            search_space['gamma'] = list(gridsearch_para['gamma']) + ['scale', 'auto']
            search_space['coef0'] = gridsearch_para['coef0']
    if clf_name == 'RandomForest':
        # 全部转为整数
        gridsearch_para_convert = convert_float_to_int(gridsearch_para)
        search_space = gridsearch_para_convert
    if clf_name == 'Logistic':
        search_space = gridsearch_para
    if clf_name == 'KNN':
        search_space = gridsearch_para
    if clf_name == 'XGBoost':
        search_space = gridsearch_para
    if clf_name == 'lightGBM':
        search_space = gridsearch_para
    if clf_name == 'Adaboost':
        search_space = gridsearch_para
    if clf_name == 'DecisionTree':
        search_space = gridsearch_para
    if clf_name == 'GBDT':
        search_space = gridsearch_para
    return search_space

In [ ]:

def select_class_model(request):
    select_child_model = request.POST.get('select_child_model').replace('task_', '')
    # gridsearch para
    gridsearch_para = {}
    grid = request.POST.get('usr_grid')
    print('usr_grid:', grid)
    if select_child_model == 'naivebayes':
        select_model_name = 'Naive Bayes'
        alpha, name = (request.POST.get('naivebayes_alpha')), request.POST.get('naivebayes_name')
        if grid == 'G': #判断是否进行gridsearch寻优，是则对获取数据进行整理
            select_model = naivebayes(name)
            grid_alpha = request.POST.get('naivebayes_alpha_grid')
            gs_para = {'alpha': grid_alpha,}
            gridsearch_para = grid_para_split(gs_para,request)
        else:
            if name == 'GaussianNB':
                select_model = naivebayes(name)
            else:
                select_model = naivebayes(name,Alpha=alpha)
    elif select_child_model == 'svm':
        select_model_name = 'SVM'
        kernel = request.POST.get('svm_kernel')
        if grid == "G":
            select_model = svm(kernels=kernel)
            grid_c,grid_degree,grid_coef,grid_gamma = request.POST.get('svm_c_grid'),\
                                       request.POST.get('svm_degree_grid'), \
                                       request.POST.get('svm_coef_grid'), \
                                       request.POST.get('svm_gamma_grid')
            gs_para = {
                'C':grid_c,
                'degree': grid_degree,
                'coef0': grid_coef,
                'gamma': grid_gamma,
            }
            gridsearch_para = grid_para_split(gs_para,request)
        else:
            c, degree, coef, gamma = float(request.POST.get('svm_c')), \
                                       request.POST.get('svm_degree'), \
                                       request.POST.get('svm_coef'), \
                                       request.POST.get('svm_gamma')
            if kernel == 'linear':
                select_model = svm(kernels=kernel, c=c)
            elif kernel == 'poly':
                degree, coef = int(degree), float(coef)
                select_model = svm(kernels=kernel, degrees=degree, c=c, coef=coef, Gamma=gamma)
            elif kernel == 'sigmoid':
                coef = float(coef)
                select_model = svm(kernels=kernel, c=c, coef=coef, Gamma=gamma)
            else:
                select_model = svm(kernels=kernel, c=c, Gamma=gamma)

    elif select_child_model == 'randomforest':
        select_model_name = 'RandomForest'
        criterion,max_features = request.POST.get('randomforest_criterion'),\
                                 request.POST.get('randomforest_max_features'),
        if grid == "G":
            select_model = randomforest(Criterion=criterion,Max_features=max_features)
            max_depth_grid, min_samples_split_grid, min_samples_leaf_grid, n_estimators_grid = \
                request.POST.get('randomforest_max_depth_grid'), \
                request.POST.get('randomforest_min_samples_split_grid'), \
                request.POST.get('randomforest_min_samples_leaf_grid'), \
                request.POST.get('randomforest_n_estimators_grid')

            gs_para = {
                'max_depth': max_depth_grid,
                'min_samples_split': min_samples_split_grid,
                'min_samples_leaf': min_samples_leaf_grid,
                'n_estimators': n_estimators_grid,
            }
            gridsearch_para = grid_para_split(gs_para, request,['int']*4)
        else:
            max_depth, min_samples_split, min_samples_leaf, n_estimators = request.POST.get('randomforest_max_depth'), \
                                               request.POST.get('randomforest_min_samples_split'), \
                                               request.POST.get('randomforest_min_samples_leaf'), \
                                               int(request.POST.get('randomforest_n_estimators'))
            max_depth, min_samples_split, min_samples_leaf, max_features = \
                surv_para_group(max_depth, min_samples_split, min_samples_leaf, max_features)
            select_model = randomforest(Criterion=criterion,Max_depth=max_depth,Min_samples_split=min_samples_split,
                                        Min_samples_leaf=min_samples_leaf,Max_features=max_features,
                                        N_estimators=n_estimators)

    elif select_child_model == 'logistic':
        select_model_name = 'Logistic'
        penalty, solver, fit_intercept = request.POST.get('logistic_penalty'), \
                                                        request.POST.get('logistic_solver'), \
                                                        bool(request.POST.get('logistic_fit_intercept')),
        if grid == 'G':
            select_model = logistic_reg(Penalty=penalty,Fit_intercept=fit_intercept, Solver=solver)
            C_grid = request.POST.get('logistic_C_grid')
            gs_para = {'C': C_grid,}
            gridsearch_para = grid_para_split(gs_para, request)
        else:
            C = request.POST.get('logistic_C')
            C = float(C)
            select_model = logistic_reg(Penalty=penalty, CC=C, Fit_intercept=fit_intercept, Solver=solver)

    elif select_child_model == 'knn':
        select_model_name = 'KNN'
        algorithm, metric, weights, n_neighbors = request.POST.get('knn_algorithm'), \
                                                                request.POST.get('knn_metric'), \
                                                                request.POST.get('knn_weights'), \
                                                                int(request.POST.get('knn_n_neighbors')),
        if grid == "G":
            select_model = kneighbors(Weight=weights,Agorithm=algorithm, Metric=metric)
            n_neighbors_grid = request.POST.get('knn_n_neighbors_grid')
            gs_para = {'n_neighbors': n_neighbors_grid, }
            gridsearch_para = grid_para_split(gs_para, request)
        else:
            n_neighbors = request.POST.get('knn_n_neighbors')
            select_model = kneighbors(Weight=weights, Agorithm=algorithm, Metric=metric,N_neighbors=n_neighbors)
    elif select_child_model == 'xgboost':
        select_model_name = 'XGBoost'
        subsample, colsample_bytree,reg_alpha, reg_lambda = float(request.POST.get('xgboost_subsample')),\
                                                            float(request.POST.get('xgboost_colsample_bytree')), \
                                                            float(request.POST.get('xgboost_reg_alpha')), \
                                                            float(request.POST.get('xgboost_reg_lambda'))
        if grid == 'G':
            select_model = xgboost(Subsample=subsample, Colsample_bytree=colsample_bytree, Reg_alpha=reg_alpha,
                                   Reg_lambda=reg_lambda)
            learning_rate_grid, n_estimators_grid, min_child_weight_grid, Gamma_grid = \
                request.POST.get('xgboost_learning_rate_grid'), \
                request.POST.get('xgboost_n_estimators_grid'), \
                request.POST.get('xgboost_min_child_weight_grid'), \
                request.POST.get('xgboost_Gamma_grid')

            gs_para = {
                'learning_rate': learning_rate_grid,
                'n_estimators': (n_estimators_grid),
                'min_child_weight':(min_child_weight_grid),
                'Gamma': Gamma_grid
            }
            gridsearch_para = grid_para_split(gs_para, request,['float','int','int','float'])
            print('gridsearch_para: ',gridsearch_para)

        else:
            learning_rate, n_estimators, min_child_weight, Gamma, = float(request.POST.get('xgboost_learning_rate')), \
                                    int(request.POST.get('xgboost_n_estimators')), \
                                    int(request.POST.get('xgboost_min_child_weight')), \
                                    float(request.POST.get('xgboost_Gamma'))
            select_model = xgboost(Learning_rate=learning_rate, N_estimators=n_estimators,
                                   Min_child_weight=min_child_weight,
                                   Subsample=subsample, Colsample_bytree=colsample_bytree, Gamma=Gamma,
                                   Reg_alpha=reg_alpha,
                                   Reg_lambda=reg_lambda)


    elif select_child_model == 'lightgbm':
        select_model_name = 'lightGBM'
        boosting_type, reg_alpha, reg_lambda, min_child_samples, subsample, = \
            request.POST.get('lightgbm_boosting_type'), \
            float(request.POST.get('lightgbm_reg_alpha')), \
            float(request.POST.get('lightgbm_reg_lambda')), \
            int(request.POST.get('lightgbm_min_child_samples')), \
            float(request.POST.get('lightgbm_subsample'))
        if grid == 'G':
            select_model = lightgbm(Boosting_type=boosting_type, Min_child_samples=min_child_samples,
                                    Reg_alpha=reg_alpha, Reg_lambda=reg_lambda,Subsample=subsample,)
            colsample_bytree_grid,learning_rate_grid, n_estimators_grid,num_leaves_grid, max_depth_grid = \
                (request.POST.get('lightgbm_colsample_bytree_grid')), \
                (request.POST.get('lightgbm_learning_rate_grid')), \
                (request.POST.get('lightgbm_n_estimators_grid')), \
                (request.POST.get('lightgbm_num_leaves_grid')), \
                (request.POST.get('lightgbm_max_depth_grid'))
            gs_para = {
                'colsample_bytree': colsample_bytree_grid,
                'learning_rate': learning_rate_grid,
                'n_estimators': n_estimators_grid,
                'num_leaves': num_leaves_grid,
                'max_depth': max_depth_grid
            }
            gridsearch_para = grid_para_split(gs_para, request,['float','float','int','int','int'])
        else:
            colsample_bytree, learning_rate, n_estimators,num_leaves, max_depth = \
                float(request.POST.get('lightgbm_colsample_bytree')), \
                float(request.POST.get('lightgbm_learning_rate')), \
                int(request.POST.get('lightgbm_n_estimators')), \
                int(request.POST.get('lightgbm_num_leaves')), \
                int(request.POST.get('lightgbm_max_depth'))
            select_model = lightgbm(Boosting_type=boosting_type,N_estimators=n_estimators,
                                    Min_child_samples=min_child_samples,Reg_alpha=reg_alpha,Reg_lambda=reg_lambda,
                                    Subsample=subsample,Colsample_bytree=colsample_bytree,Num_leaves=num_leaves,
                                    Max_depth=max_depth,Learning_rate=learning_rate)
    elif select_child_model == 'adaboost':
        select_model_name = 'Adaboost'
        algorithm = request.POST.get('adaboost_algorithm')
        if grid == 'G':
            select_model = adaboost( Algorithm=algorithm,)
            n_estimators_grid, learning_rate_grid, max_depth_grid = (request.POST.get('adaboost_n_estimators_grid')), \
                                                                (request.POST.get('adaboost_learning_rate_grid')), \
                                                                (request.POST.get('adaboost_max_depth_grid'))
            gs_para = {
                'learning_rate': learning_rate_grid,
                'n_estimators': n_estimators_grid,
                'max_depth': max_depth_grid
            }
            gridsearch_para = grid_para_split(gs_para, request, ['int', 'float', 'int'])
        else:
            n_estimators, learning_rate, max_depth = int(request.POST.get('adaboost_n_estimators')), \
                                                     float(request.POST.get('adaboost_learning_rate')), \
                                                     int(request.POST.get('adaboost_max_depth'))
            select_model = adaboost(N_estimators=n_estimators, Learning_rate=learning_rate,Algorithm=algorithm,Max_depth=max_depth)
    elif select_child_model == 'decisiontree':
        select_model_name = 'DecisionTree'
        criterion, splitter,max_features =  request.POST.get('decisiontree_criterion'),\
                                            request.POST.get('decisiontree_splitter'), \
                                            request.POST.get('decisiontree_max_features')
        if grid == 'G':
            select_model = decisiontree(Criterion=criterion, Splitter=splitter, Max_features=max_features)
            max_depth_grid, min_samples_split_grid, min_samples_leaf_grid = \
                request.POST.get('decisiontree_max_depth_grid'), \
                request.POST.get('decisiontree_min_samples_split_grid'), \
                request.POST.get('decisiontree_min_samples_leaf_grid')
            gs_para = {
                'max_depth': max_depth_grid,
                'min_samples_split': min_samples_split_grid,
                'min_samples_leaf': min_samples_leaf_grid
            }
            gridsearch_para = grid_para_split(gs_para, request,['int','float','int'])
        else:
            max_depth, min_samples_split, min_samples_leaf = \
                                                            request.POST.get('decisiontree_max_depth'), \
                                                            request.POST.get('decisiontree_min_samples_split'), \
                                                            request.POST.get('decisiontree_min_samples_leaf')
            max_depth, min_samples_split, min_samples_leaf, max_features = \
                surv_para_group(max_depth, min_samples_split, min_samples_leaf, max_features)
            select_model = decisiontree(Criterion=criterion,Splitter=splitter,Max_depth=max_depth,
                                        Min_samples_split=min_samples_split,Min_samples_leaf=min_samples_leaf,
                                        Max_features=max_features)
    else:
        select_model_name = 'GBDT'
        criterion, loss, subsample, max_features, = request.POST.get('gbdt_criterion'), \
                         request.POST.get('gbdt_loss'), \
                         float(request.POST.get('gbdt_subsample')), \
                         request.POST.get('gbdt_max_features'),
        if grid == 'G':
            select_model = gbdt(Loss=loss,Subsample=subsample,Criterion=criterion,Max_features=max_features)
            learning_rate_grid,min_samples_split_grid, min_samples_leaf_grid, max_depth_grid,n_estimators_grid =  \
                         (request.POST.get('gbdt_learning_rate_grid')), \
                         request.POST.get('gbdt_min_samples_split_grid'), \
                         request.POST.get('gbdt_min_samples_leaf_grid'), \
                         request.POST.get('gbdt_max_depth_grid'), \
                         (request.POST.get('gbdt_n_estimators_grid')),
            gs_para = {
                'learning_rate': learning_rate_grid,
                'min_samples_split': min_samples_split_grid,
                'min_samples_leaf': min_samples_leaf_grid,
                'max_depth':max_depth_grid,
                'n_estimators':n_estimators_grid

            }
            gridsearch_para = grid_para_split(gs_para, request,['float','int','int','int','int'])
        else:
            learning_rate,min_samples_split, min_samples_leaf, max_depth,n_estimators =  \
                         float(request.POST.get('gbdt_learning_rate')), \
                         request.POST.get('gbdt_min_samples_split'), \
                         request.POST.get('gbdt_min_samples_leaf'), \
                         request.POST.get('gbdt_max_depth'), \
                         int(request.POST.get('gbdt_n_estimators')),
            max_depth, min_samples_split, min_samples_leaf, max_features = \
                surv_para_group(max_depth, min_samples_split, min_samples_leaf, max_features)
            select_model = gbdt(Loss=loss,Learning_rate=learning_rate,Subsample=subsample,
                                Criterion=criterion,Min_samples_split=min_samples_split,Max_features=max_features,
                                Min_samples_leaf=min_samples_leaf,Max_depth=max_depth,Validation_fraction=validation_fraction,
                                N_estimators=n_estimators)
    return select_model, select_model_name,gridsearch_para

In [ ]:
def grid_para_split(paras,request,type=['float']*6):
    from django.contrib import messages
    para_lsts = {}
    i = 0
    for para_key,para in paras.items():
        if para == None:
            continue
        if len(para.split(":")) != 3:
            title = 'Invalid input.Please use the "from:to:step" format for parameter input.'
            messages.success(request, title)
            return render(request, "analysis.html")
        if type[i] == 'int':
            try:
                para_from = int(para.split(":")[0])
                para_to = int(para.split(":")[1])
                para_step = int(para.split(":")[2])
            except Exception as e:
                print(e)
                title = 'Invalid input.Please use the "1:10:1" format for parameter input.'
                messages.success(request, title)
                return render(request, "analysis.html")
            para_from,para_to,para_step = int(para_from),int(para_to),int(para_step)
            para_lst = np.arange(para_from, para_to+1, para_step)
        else:
            try:
                para_from = float(para.split(":")[0])
                para_to = float(para.split(":")[1])
                para_step = float(para.split(":")[2])
            except Exception as e:
                print(e)
                title = 'Invalid input.Please use the "1:10:1" format for parameter input.'
                messages.success(request, title)
                return render(request, "analysis.html")
            para_lst = np.arange(para_from, para_to+0.01, para_step) #左闭右开
        para_lsts[para_key] = para_lst
        i = i + 1
    return para_lsts

In [ ]:
def convert_float_to_int(obj):
    if isinstance(obj, dict):
        return {key: convert_float_to_int(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_float_to_int(item) for item in obj]
    elif isinstance(obj, np.ndarray):
        return obj.astype(int)
    elif isinstance(obj, float):
        return int(obj)
    else:
        return obj